In [ ]:
import numpy as np
import nbimporter
import m00_helper as helper
import m01_profiles as profiles

### Weight reps based on default weighting and delegations

In [ ]:
# def weight_reps(delegator_ids, rep_ids, election_profile, 
#                 default_style:str, delegation_style:str, weighting_function:str):
#     '''Sets default weights, updates weights given by voters based on delegation, then aggregates to assign weights to reps'''
#     default_given_weights = set_default_given_weights(default_style, election_profile)
#     given_weights = delegate(default_given_weights, delegator_ids, rep_ids, delegation_style, election_profile)
#     rep_weights = aggregate_weights(given_weights, weighting_function)
#     return rep_weights

In [ ]:
def set_default_given_weights(default_style:str, election_profile:np.ndarray, cand_ids, rep_ids, normalized:bool = True):
    '''returns matrix of size n_voters x n_cands'''
    n_voters, n_cands = election_profile.shape[0], election_profile.shape[1]
    if len(cand_ids) != n_cands:
        raise ValueError('setting default given weights:\ncand_ids len needs to match the number of columns in the election_profile')
    
    if default_style == 'uniform':
        default_given_weights = np.ones_like(election_profile)
    elif default_style == 'borda_ordermap':
        default_given_weights = n_cands - election_profile #election profile needs to be an ordermap, not an order!!
    elif default_style == 'approvals':
        default_given_weights = election_profile
    elif default_style == 'scores':
        default_given_weights = election_profile
    else:
        raise ValueError(f'The default weighting style {default_style} is not implemented.')
        
    #Only have positive weights on elected reps, not all cands, set unelected cands to be given weight 0
    for cand in cand_ids:
        if cand not in rep_ids:
            for voter in range(n_voters):
                default_given_weights[voter,cand] = 0

    if normalized == True:
        #normalize so weights given by each voter sum to one
        return helper.normalize_rows_2D(default_given_weights)
    else:
        return default_given_weights

In [ ]:
def best_k_delegation(default_given_weights, k, delegator_ids, rep_ids, voter_cand_distances, normalized=True):
    n_reps = len(rep_ids)
    given_weights = default_given_weights
    for voter in delegator_ids:
        given_weights[voter] = np.zeros(n_reps)
        voter_distances = voter_cand_distances[voter]
        order = profiles.distances_to_orders_one_voter(voter_distances) #includes random tie breaking
        delegation_count = 0
        
        for rep in order:
            if rep in rep_ids:
                given_weights[voter][rep] = 1
                delegation_count += 1
            if delegation_count >= k:
                break
            
    if normalized == True:
        #normalize so weights given by each voter sum to one
        given_weights = helper.normalize_rows_2D(given_weights)
        
    return given_weights
    

In [ ]:
def approval_delegation(default_given_weights, delegator_ids, rep_ids, approval_profile, normalized=True):
    #for all delegating voters, replace their given weights with their approvals from the election, among the elected reps
    n_cands = approval_profile.shape[1]
    rep_indicator = [1 if rep in rep_ids else 0 for rep in range(n_cands)]
    given_weights = default_given_weights
    
    for voter in delegator_ids:
        given_weights[voter] = approval_profile[voter] * rep_indicator
        
    if normalized == True:
        #normalize so weights given by each voter sum to one
        given_weights = helper.normalize_rows_2D(given_weights)
    
    return given_weights



array([[0.        , 1.        , 0.        ],
       [0.33333333, 0.33333333, 0.33333333]])

In [ ]:
def decisive_delegation(default_given_weights, delegator_ids, rep_ids, 
                        voter_issues_profile, cand_issues_profile, issue):
    n_voters, n_cands, n_reps = default_given_weights.shape[0], default_given_weights.shape[1], len(rep_ids)
    given_weights = default_given_weights
    for v in delegator_ids:
        given_weights[v] = np.zeros(n_cands)
        found = False
        for rep in rep_ids:
            if found == True:
                continue
            if voter_issues_profile[v][issue] == cand_issues_profile[rep][issue]:
                given_weights[v][rep] = 1
                found = True
        if found == False: #default delegation behavior if none of the reps agree with the voter on that issue
            for rep in rep_ids:
                given_weights[v][rep] = 1.0 / n_reps
    return given_weights

In [ ]:
def aggregate_weights(given_weights):
    return np.sum(given_weights, axis=0)

### Determine which voters delegate

In [ ]:
def choose_delegators(n_voters, n_delegators):
    '''randomly select a fixed number of voters as delegators'''
    return  np.random.choice(n_voters, n_delegators, replace=False)

In [ ]:
def independent_delegators(delegation_probs):
    '''Determines which voters delegate where each voter has their own given probability of delegating
    e.g. based on preference intensity
    
    
    RETURNS
    -------
    Indicator array of elems in {0,1} of length len(delegation_probs)
    
    
    NOTES
    ------
    To be used for determining which voters delegate when they all decide whether to delegate with some independent probability
    
    '''
    return [np.random.binomial(1, p) for p in delegation_probs]

# TESTING

In [ ]:
approval_delegation(np.array([[1,1,1],[1,1,1]]), [0], [1,2], np.array([[1,1,0],[1,1,1]]))

In [ ]:
independent_delegators([0, 0.5, 0.5])

In [ ]:
aggregate_weights([[1,0,0],[0.5,0.5,1]])